# GLD Example

This notebook demonstrates how to generate groundwaterlevel research (GLD) related sourcedocuments for data exchange with the bronhouderportaal (BRO). Also, it is shown how to validate these documents and upload them with the bronhouderportaal api. 

In [3]:
import bro_exchange as brx
from getpass import getpass

Define the token used for authentification with the Bronhouderportaal API below. Note: a token generated in the demo environment should be used.

In [5]:
user = input('user:')
key = getpass('pass:')

acces_token_bro_portal = {
    'user':user,
    'pass':key
    }

## 1. Registrate a new GLD

### 1.1. Generate registration request xml document

Get GLD data:

In [6]:
monitoringpoints = [{'broId':'brx000000016118',
                     
                   'tubeNumber':1}]

groundwatermonitoringnets = [{'broId':'GMN000000000083'},
                             {'broId':'GMN000000000084'}]

#%% Inputdata for brx_Construction sourcedocument

srcdocdata =  {'objectIdAccountableParty':'Meetpunt1',
            'groundwaterMonitoringNets':groundwatermonitoringnets, #     
            'monitoringPoints':monitoringpoints,        #
            }    


Generate registartion request (GLD_StartRegistration) xml file:

In [8]:
# NOTE: DEFINE OUTPUT DIRECTORY:
reg = brx.gld_registration_request(srcdoc='GLD_StartRegistration', requestReference = '10_GLD_StartRegistration', deliveryAccountableParty = '27376655', qualityRegime = 'IMBRO', srcdocdata=srcdocdata)
reg.generate()
reg.write_request(output_dir = 'output\gld', filename = 'registration_request_gld_startregistration_test.xml')


### 1.2. Validate registration request xml document

Validate the generated sourcedocument (by means of bronhouderportaal api). In this case, the demo environment of the bronhouderportaal is used.

In [10]:
payload = reg.request


Check if there are some errors:

In [11]:
#payload = reg.request
validation_info = brx.validate_request(payload, acces_token_bro_portal, demo=True)

print(validation_info['status']+'\n')
try:
    
    print('ERRORS:')
    for error in validation_info['errors']:
        print(error+'\n')
except:
    None


NIET_VALIDE

ERRORS:
GMW-monitoringbuis.broId (GroundwaterMonitoringTube.broId) = brx000000016118 heeft geen correct formaat: moet zijn GMWNNNNNNNNNNNN.

GMW-monitoringbuis (GroundwaterMonitoringTube) ( brx000000016118, 1 ) [BRO-ID, buisnummer] is niet correct. De combinatie van Monitoringbuis en buisnummer is onbekend.

Er zijn 1 of meer fouten geconstateerd in het brondocument.



The xml document seems to be valid

## 2. Add groundwaterlevel data to GLD

### 2.1. Generating registration request xml document

Get groundwaterlevel data (in this case three seperate timeseries for the same GLD, that subsequently contain partly validated, validated and verification measurements)

In [12]:
# Parameters voorlopig & volledig beoordeeld:
metadata_parameters_1= {'principalInvestigator':{'europeanCompanyRegistrationNumber':'DEB8537.HRB66039'}, # kvk principalinvestigator
            'observationType':'reguliereMeting'
             }

procedure_parameters_1 = {'airPressureCompensationType':'KNMImeting',
                        'evaluationProcedure':'oordeelDeskundige',
                        'measurementInstrumentType':'akoestischeSensor'   
                        }

# Parameters controlemeting:
metadata_parameters_2= {'principalInvestigator':'27376655',
            'observationType':'controlemeting'
             }



procedure_parameters_2 = {'evaluationProcedure':'oordeelDeskundige',
                        'measurementInstrumentType':'analoogPeilklokje'   
                        }

# Timeseriesdata (Eenheid is verplicht meter tov NAP):
results_voorlopig = [{'time':'2018-01-07T08:14:38+01:00','value':-4.345,'metadata':{'StatusQualityControl':'goedgekeurd','interpolationType':'Discontinuous'}},#'censoredReason':}}
                    {'time':'2018-01-21T10:01:52+01:00','value':'None','metadata':{'StatusQualityControl':'afgekeurd','interpolationType':'Discontinuous','censoringLimitvalue':0,'censoredReason':'BelowDetectionRange'}} ,                      
                    {'time':'2018-01-28T16:58:07+01:00','value':-4.788,'metadata':{'StatusQualityControl':'goedgekeurd','interpolationType':'Discontinuous'}},#'censoredReason':}}                        
                    ]

results_controle = [{'time':'2018-04-15T08:05:42+02:00','value':-4.350,'metadata':{'StatusQualityControl':'goedgekeurd','interpolationType':'Discontinuous'}}
                    ]

results_volledig_beoordeeld = [{'time':'2018-01-07T08:14:38+01:00','value':-4.345,'metadata':{'StatusQualityControl':'goedgekeurd','interpolationType':'Discontinuous'}},#'censoredReason':}}
                    {'time':'2018-01-21T10:01:52+01:00','value':-4.987,'metadata':{'StatusQualityControl':'goedgekeurd','interpolationType':'Discontinuous'}},#'censoredReason':}}                        
                    {'time':'2018-01-28T16:58:07+01:00','value':-4.788,'metadata':{'StatusQualityControl':'goedgekeurd','interpolationType':'Discontinuous'}},#'censoredReason':}}                        
                    ]

    
# Input arguments sourcedocs additions (voorlopig, volledigBeoordeeld & controlemeting)  
arglist_voorlopig =  {'metadata':{'status':'voorlopig','parameters':metadata_parameters_1},
                        'procedure':{'parameters':procedure_parameters_1}, 
                        'resultTime':'2018-01-28T16:58:07+01:00',
                        'result':results_voorlopig, 
                        }    

arglist_controle =  {'metadata':{'parameters':metadata_parameters_2}, 
                        'procedure':{'parameters':procedure_parameters_2},
                        'resultTime':'2018-04-15T08:05:42+02:00',
                        'result':results_controle, 
                        }  

arglist_volledig_beoordeeld =  {'metadata':{'status':'volledigBeoordeeld','parameters':metadata_parameters_1},
                                    'procedure':'_047e1b31-5406-4c68-af73-303e3f057881',
                                    'resultTime':'2018-05-28T16:58:07+01:00', # Resultaat volledig beoordeeld is als de validatie is afgerond
                                    'result':results_volledig_beoordeeld, 
                                    }   


Create and validate the registration requests (gld_additions):

In [13]:
# voorlopig
print("Registration request partly validated series:\n")
reg_voorlopig = brx.gld_registration_request(srcdoc='GLD_Addition', requestReference = '11_GLD_Addition_sensorisch', deliveryAccountableParty = '27376655', qualityRegime = 'IMBRO',broId='GLD000000000153', srcdocdata=arglist_voorlopig)
reg_voorlopig.generate()
reg_voorlopig.write_request(output_dir = 'output\gld', filename = 'registration_request_gld_addition_voorlopig_test.xml')

payload = reg_voorlopig.request

print(validation_info['status']+'\n')
try:
    
    print('ERRORS:\n')
    for error in validation_info['errors']:
        print(error+'\n')
except:
    None
print('-------------------\n')

# controle
print("Registration request verification series:\n")
reg_controle = brx.gld_registration_request(srcdoc='GLD_Addition', requestReference = '11_GLD_Addition_sensorisch', deliveryAccountableParty = '27376655', qualityRegime = 'IMBRO',broId='GLD000000000153', srcdocdata=arglist_controle)
reg_controle.generate()
reg_controle.write_request(output_dir = 'output\gld', filename = 'registration_request_gld_addition_controle_test.xml')

payload = reg_voorlopig.request

print(validation_info['status']+'\n')
try:
    
    print('ERRORS:\n')
    for error in validation_info['errors']:
        print(error+'\n')
except:
    None
print('-------------------\n')
# volledig
print("Registration request validated series:\n")
reg_volledig = brx.gld_registration_request(srcdoc='GLD_Addition', requestReference = '11_GLD_Addition_sensorisch', deliveryAccountableParty = '27376655', qualityRegime = 'IMBRO',broId='GLD000000000153', srcdocdata=arglist_volledig_beoordeeld)
reg_volledig.generate()
reg_volledig.write_request(output_dir = 'output\gld', filename = 'registration_request_gld_addition_volledig_test.xml')

payload = reg_voorlopig.request

print(validation_info['status']+'\n')
try:
    
    print('ERRORS:\n')
    for error in validation_info['errors']:
        print(error+'\n')
except:
    None
print('-------------------\n')


Registration request partly validated series:

NIET_VALIDE

ERRORS:

GMW-monitoringbuis.broId (GroundwaterMonitoringTube.broId) = brx000000016118 heeft geen correct formaat: moet zijn GMWNNNNNNNNNNNN.

GMW-monitoringbuis (GroundwaterMonitoringTube) ( brx000000016118, 1 ) [BRO-ID, buisnummer] is niet correct. De combinatie van Monitoringbuis en buisnummer is onbekend.

Er zijn 1 of meer fouten geconstateerd in het brondocument.

-------------------

Registration request verification series:

NIET_VALIDE

ERRORS:

GMW-monitoringbuis.broId (GroundwaterMonitoringTube.broId) = brx000000016118 heeft geen correct formaat: moet zijn GMWNNNNNNNNNNNN.

GMW-monitoringbuis (GroundwaterMonitoringTube) ( brx000000016118, 1 ) [BRO-ID, buisnummer] is niet correct. De combinatie van Monitoringbuis en buisnummer is onbekend.

Er zijn 1 of meer fouten geconstateerd in het brondocument.

-------------------

Registration request validated series:

NIET_VALIDE

ERRORS:

GMW-monitoringbuis.broId (Groundwate